# EDA for NLP

In this notebook, Arabic text is converted to English text.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#loading the csv file
missing_data = pd.read_csv('/content/drive/MyDrive/1_fall_23/549 ml spark/Project/output1.csv')
missing_data.head()

,case_id,case_code,counts_photos,counts_enhanced_photos,counts_posts,post_ids,post_text,ocr_text
0,003e1cfe-434f-4bc8-9cc0-61dddf22abad,23040412,1,1,1,8b9cf138-5602-436a-b784-5c60af8a4f5e,[خالد مصطفي حسن عرفان \nعمره الان حوالي 12 سنه...,"[May be an image of child, smiling and pie]"
1,0175541d-aff4-4b26-8095-24c1caca449c,2305019,1,1,1,8d318381-f68e-43bc-a298-19186be7828d,[تحديث: الحمد لله علي رجع لاهله حمد الله علي ا...,"[May be a selfie of 1 person, beard, smiling a..."
2,01b2c56d-878f-49f9-b338-e6dae8ebe357,23051710,1,2,1,1925f483-e4d6-459a-9ac7-26621290b4da,[الراجل دا بيتسول بالطفل في منطقه المهندسين جا...,"[May be an image of 2 people, people smiling a..."
3,023e7ae8-d5f2-4091-914c-008bc4b6e6d5,2304091,2,4,2,"2ee3152a-450a-4ea9-a160-2665c6bf3dc0, 4f08d88a...",[تحديث : الحمد لله السيد رجع لاهله بالسلامه\n-...,"[May be an image of one or more people], [May ..."
4,02546fc1-80df-4e9c-81d4-a72509734a23,2304078,1,2,1,8e834d44-872e-4e57-88ca-c3d4594e99a8,[هذا الرجل موجود بتاريخ 6 ابريل 2023 في ابو شل...,"[May be an image of ‎one or more people, beard..."


In [ ]:
column_names = missing_data.columns

# Print the column names
print(column_names)

Index(['case_id', 'case_code', 'counts_photos', 'counts_enhanced_photos',
       'counts_posts', 'post_ids', 'post_text', 'ocr_text'],
      dtype='object')


In [ ]:
missing_data.drop(columns=['post_ids'], inplace=True)
missing_data.head()

,case_id,case_code,counts_photos,counts_enhanced_photos,counts_posts,post_text,ocr_text
0,003e1cfe-434f-4bc8-9cc0-61dddf22abad,23040412,1,1,1,[خالد مصطفي حسن عرفان \nعمره الان حوالي 12 سنه...,"[May be an image of child, smiling and pie]"
1,0175541d-aff4-4b26-8095-24c1caca449c,2305019,1,1,1,[تحديث: الحمد لله علي رجع لاهله حمد الله علي ا...,"[May be a selfie of 1 person, beard, smiling a..."
2,01b2c56d-878f-49f9-b338-e6dae8ebe357,23051710,1,2,1,[الراجل دا بيتسول بالطفل في منطقه المهندسين جا...,"[May be an image of 2 people, people smiling a..."
3,023e7ae8-d5f2-4091-914c-008bc4b6e6d5,2304091,2,4,2,[تحديث : الحمد لله السيد رجع لاهله بالسلامه\n-...,"[May be an image of one or more people], [May ..."
4,02546fc1-80df-4e9c-81d4-a72509734a23,2304078,1,2,1,[هذا الرجل موجود بتاريخ 6 ابريل 2023 في ابو شل...,"[May be an image of ‎one or more people, beard..."


In [ ]:
#printing ocr_text
ocr_text_column = missing_data['ocr_text']

# Print the entire content of the first 50 strings in the 'ocr_text' column
for i, text in enumerate(ocr_text_column[:10], start=1):
    print(f"String {i}:\n{text}\n")


String 1:
[May be an image of child, smiling and pie]

String 2:
[May be a selfie of 1 person, beard, smiling and text]

String 3:
[May be an image of 2 people, people smiling and text]

String 4:
[May be an image of one or more people], [May be an image of child, hair and smiling]

String 5:
[May be an image of ‎one or more people, beard, people standing, turtleneck, snowsuit and ‎text that says '‎أطف مفق‎'‎‎]

String 6:
[May be an image of one or more people, hair, people smiling and text]

String 7:
[No photo description available.]

String 8:
[May be a meme of 1 person and text]

String 9:
[No photo description available.]

String 10:
[May be a selfie of 4 people, beard, hair, people smiling and text]



Note - from OCR text, we get some info like 'beard' sometimes. Then from the translations, we'll get gender info. We'll even match these 2, otherwise the gender info will be used. Then we will see how many of them are male and females. Then we'll use a gender predictor and feed it images, match with case IDs to see how many genders we have accurately predicted. Let's see the number of different cases, if they are below 50, we can also have a look and establish ground truth.

Next step is to extract the gender information from the translation.

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import pandas as pd
from googletrans import Translator

In [ ]:
translator = Translator()

In [ ]:
# Translate the first 5 rows of the 'post_text' column and store in a new DataFrame
num_rows_to_translate = 1765
translated_data = missing_data.head(num_rows_to_translate).copy()
translated_data['post_text' + '_English'] = translated_data['post_text'].apply(lambda x: translator.translate(x, src='ar', dest='en').text)

# Save the translated data to a new CSV file
#output_file = 'translated_output.csv'
#translated_data.to_csv(output_file, index=False)


In [ ]:
translated_data.tail()

,case_id,case_code,counts_photos,counts_enhanced_photos,counts_posts,post_text,ocr_text,post_text_English
1759,4d6fac58-8c73-4d22-b9f1-bcb35a9b54bb,2310232,1,0,1,[محمود محمد البس\n13 سنه \nتغيب يوم 22 اكتوبر ...,"[May be a meme of 1 person, smiling, costume a...",[Mahmoud Muhammad Albas\n13 years\nHe was abse...
1760,de46c16a-436a-49de-bd1c-a4fad9e7009b,2211064,1,0,1,[محمد عبد العال عبد الرازق ادم\nمواليد 1976\nت...,[No photo description available.],[Mohamed Abdel-Al Abdel-Razzaq Adam\nBorn in 1...
1761,96591f55-5ef6-4e6f-8282-baba66fa03aa,2211066,1,0,1,[محمد عطيه عتمان\nمواليد 2000\nتغيب في سبتمبر ...,[No photo description available.],[Mohamed Atiya Etman\nBorn in 2000\nHe disappe...
1762,2f638149-a09f-4a9e-b614-0a305d7ea5b2,1910097,2,0,1,[نداء خاصة لاهالي السعوديه \nغدير عبد الله غان...,"[No photo description available.], [No photo d...",[A special appeal to the people of Saudi Arabi...
1763,6aa22b01-84cd-4bc0-9f1e-983c3d767cda,2102025,2,0,1,[مجدي سعد فهيم\nمواليد 1995 \nتغيب في يونيو 19...,"[No photo description available.], [No photo d...",[Magdy Saad Fahim\nBorn in 1995\nHe was absent...


In [ ]:
# Save the translated data to a new CSV file
output_file = 'translated_data.csv'
translated_data.to_csv(output_file, index=False)


In [ ]:
post_texts = translated_data['post_text_English']

In [ ]:
post_texts

0       [Khaled Mustafa Hassan Irfan\nHe is now about ...
1       [Update: Thank God he returned to his family, ...
2       [This man is begging with children in the Moha...
3       [Update: Praise be to God, the master returned...
4       [This man was present on April 6, 2023 in Abu ...
                              ...                        
1759    [Mahmoud Muhammad Albas\n13 years\nHe was abse...
1760    [Mohamed Abdel-Al Abdel-Razzaq Adam\nBorn in 1...
1761    [Mohamed Atiya Etman\nBorn in 2000\nHe disappe...
1762    [A special appeal to the people of Saudi Arabi...
1763    [Magdy Saad Fahim\nBorn in 1995\nHe was absent...
Name: post_text_English, Length: 1764, dtype: object